In [ ]:

from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# re-size all the images
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/MyDrive/chest_xray/train'
valid_path = '/content/drive/MyDrive/chest_xray/test'


In [ ]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94781440/94765736 [==============================] - 1s 0us/step


Making all the layers non traininable and droping out 1st layer so that we can use our own image size

In [ ]:
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/chest_xray/train/*')

In [ ]:
x = Flatten()(resnet.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)
 
# creating a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Data augmentation

In [ ]:


 
# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator
 
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
 
test_datagen = ImageDataGenerator(rescale = 1./255)


########################################################Generating the batches of augmentated data#############################################################


# providing the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
 

 
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


Training and evaluating the model

In [ ]:
# fiting the model
 
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5, #10,15,20
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)


Epoch 1/5
163/163 [==============================] - 809s 5s/step - loss: 0.5872 - accuracy: 0.7962 - val_loss: 0.4184 - val_accuracy: 0.7981
Epoch 2/5
163/163 [==============================] - 244s 1s/step - loss: 0.4122 - accuracy: 0.8372 - val_loss: 0.3421 - val_accuracy: 0.8574
Epoch 3/5
163/163 [==============================] - 230s 1s/step - loss: 0.3022 - accuracy: 0.8737 - val_loss: 0.3102 - val_accuracy: 0.8734
Epoch 4/5
163/163 [==============================] - 236s 1s/step - loss: 0.2956 - accuracy: 0.8829 - val_loss: 0.3051 - val_accuracy: 0.8750
Epoch 5/5
163/163 [==============================] - 225s 1s/step - loss: 0.2769 - accuracy: 0.8882 - val_loss: 0.3296 - val_accuracy: 0.8718
